In [ ]:
import gzip
import json

file_path = 'dataset0.json.gz'

objs = []
with gzip.open(file_path) as f:
            for line in f:
               objs.append(json.loads(line))

In [ ]:
objs

In [1]:
import gzip
import json
import pandas as pd

# Read dataset0.json.gz file
objs=[]

with gzip.open("dataset0.json.gz", 'rt') as f:
            for line in f:
               objs.append(json.loads(line))

In [3]:
# Read data.info table
data = pd.read_csv("data.info")
data = pd.DataFrame(data)


rows = []
for i in range(len(objs)):
    for transcript_id, inner_dict in objs[i].items():
        for position, inner_list in inner_dict.items():
            for necleotides, sublist in inner_list.items():
                for subsublist in sublist:
                    rows.append([transcript_id, position, necleotides] + subsublist)


df = pd.DataFrame(rows, columns=["transcript_id", "transcript_position", "combined nucleotides", "dwelling_time1", "sd1", "mean1", "dwelling_time2", "sd2", "mean2", "dwelling_time3", "sd3", "mean3"])

df['transcript_position'] = df['transcript_position'].astype(int)
data['transcript_position'] = data['transcript_position'].astype(int)

df = pd.merge(df, data, on=['transcript_id', 'transcript_position'], how='inner')

In [13]:
df

,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,sd3,mean3,gene_id,label
0,ENST00000000233,244,AAGACCA,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,ENSG00000004059,0
1,ENST00000000233,244,AAGACCA,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,ENSG00000004059,0
2,ENST00000000233,244,AAGACCA,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,ENSG00000004059,0
3,ENST00000000233,244,AAGACCA,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,ENSG00000004059,0
4,ENST00000000233,244,AAGACCA,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,ENSG00000004059,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11027101,ENST00000641834,1693,TTGACAT,0.00418,7.49,108.0,0.00564,10.20,116.0,0.01000,2.01,76.4,ENSG00000167747,0
11027102,ENST00000641834,1693,TTGACAT,0.00664,1.91,109.0,0.00598,12.30,110.0,0.01760,2.61,74.6,ENSG00000167747,0
11027103,ENST00000641834,1693,TTGACAT,0.00721,4.58,105.0,0.00398,6.58,113.0,0.00316,2.28,85.3,ENSG00000167747,0
11027104,ENST00000641834,1693,TTGACAT,0.00266,2.33,109.0,0.00913,10.40,108.0,0.00664,4.44,76.8,ENSG00000167747,0


In [5]:
df['label'].value_counts()

label
0    10527532
1      499574
Name: count, dtype: int64

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

combined = df 
X = combined.drop(['label','transcript_id','combined nucleotides','gene_id'], axis=1)
y = combined['label']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from keras.models import Sequential
from keras.layers import Dense

# Random Forest
clf_rf = RandomForestClassifier(random_state=42)
clf_rf.fit(X_train, y_train)
y_pred_rf = clf_rf.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

Random Forest Accuracy: 0.9581971160167986


In [16]:
# Probability Scores
prob_rf = clf_rf.predict_proba(X_test)

In [11]:
# XGBoost
clf_xgb = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
clf_xgb.fit(X_train, y_train)
y_pred_xgb = clf_xgb.predict(X_test)
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))

XGBoost Accuracy: 0.9557381761857822


In [18]:
# Probability Scores
prob_xgb = clf_xgb.predict_proba(X_test)

In [14]:
# Generate evaluations
models = {'Random Forest': y_pred_rf, 'XGBoost': y_pred_xgb}
for model_name, y_pred in models.items():
    print(f"Evaluations for {model_name}:")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))

Evaluations for Random Forest:
Accuracy: 0.9581971160167986
Confusion Matrix:
 [[2102959    2854]
 [  89339   10270]]
Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98   2105813
           1       0.78      0.10      0.18     99609

    accuracy                           0.96   2205422
   macro avg       0.87      0.55      0.58   2205422
weighted avg       0.95      0.96      0.94   2205422

Evaluations for XGBoost:
Accuracy: 0.9557381761857822
Confusion Matrix:
 [[2102302    3511]
 [  94105    5504]]
Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98   2105813
           1       0.61      0.06      0.10     99609

    accuracy                           0.96   2205422
   macro avg       0.78      0.53      0.54   2205422
weighted avg       0.94      0.96      0.94   2205422



In [15]:
from sklearn.metrics import roc_auc_score, average_precision_score

for model_name, y_pred in models.items():
    print(f'ROC AUC Score: {roc_auc_score(y_test, y_pred)}')
    print(f'Precision-Recall AUC Score: {average_precision_score(y_test, y_pred)}')


ROC AUC Score: 0.5508739186102262
Precision-Recall AUC Score: 0.1211906947076335
ROC AUC Score: 0.5267943808071914
Precision-Recall AUC Score: 0.076405756778201
